In [21]:
import os
%pwd

'D:\\Deep Learning\\Tensorflow & Keras\\deploy2\\project3'

In [22]:
os.chdir(r"D:\Deep Learning\Tensorflow & Keras\deploy2\project3")

In [23]:
%pwd

'D:\\Deep Learning\\Tensorflow & Keras\\deploy2\\project3'

In [24]:
from dataclasses import dataclass
from pathlib import Path

In [25]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_augmentation: bool
    params_image_size: list

In [26]:
from TomatoDiseases.constant import *
from TomatoDiseases.utils.common import *
from TomatoDiseases.entity.config_entity import *
import tensorflow as tf

In [27]:
class ConfigManager:
    def __init__(self,
                 config_filepath= CONFIG_FILE_PATH,
                 params_filepath= PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        checkpoint_dir = os.path.dirname(config.checkpoint_filepath)
        create_directories([Path(checkpoint_dir),
                            Path(config.tensorboard_dir)])
        
        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir= config.root_dir,
            tensorboard_dir= config.tensorboard_dir,
            checkpoint_filepath= config.checkpoint_filepath
        )
        
        return prepare_callbacks_config


    def get_model_training_config(self) -> ModelTrainerConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir)
        create_directories([Path(training.root_dir)])

        training_config = ModelTrainerConfig(
            root_dir= Path(training.root_dir),
            trained_model_path= Path(training.trained_model_path),
            updated_base_model_path= Path(prepare_base_model.updated_base_model_path),
            training_data= Path(training_data),
            params_epochs= params.EPOCHS,
            params_batch_size= params.BATCH_SIZE,
            params_augmentation= params.AUGMENTATION,
            params_image_size= params.IMAGE_SIZE
        )

        return training_config


In [28]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
from pathlib import Path
from TomatoDiseases.components.prepare_callbacks import PrepareCallbacks

In [29]:
class Training:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            filepath= self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.2
            )
        
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = 'bilinear'
        )
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory= self.config.training_data,
            subset='validation',
            shuffle= False,
            **dataflow_kwargs
        )
        
        if self.config.params_augmentation:
            train_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range= 40,
                horizontal_flip= True,
                width_shift_range= 0.2,
                height_shift_range= 0.2,
                shear_range= 0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        
        else:
            train_data_generator = valid_datagenerator
        
        self.train_generator = train_data_generator.flow_from_directory(
            directory= self.config.training_data,
            subset='training',
            shuffle=True,
            **dataflow_kwargs
        )
        
    @staticmethod
    def save_model(path: Path,
                   model: tf.keras.models):
        model.save(path)
        
    
    def train(self, callbacks_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.valid_steps = self.valid_generator.samples // self.valid_generator.batch_size
        
        self.model.fit(
            self.train_generator,
            epochs = self.config.params_epochs,
            steps_per_epoch = self.steps_per_epoch,
            validation_steps = self.valid_steps,
            validation_data = self.valid_generator,
            callbacks = callbacks_list
        )
        
        self.save_model(
            path= self.config.trained_model_path,
            model=self.model
        )

In [30]:
try:
    config = ConfigManager()
    model_trainer_callbacks_config = config.get_callbacks_config()
    model_trainer_callbacks_config = PrepareCallbacks(config=model_trainer_callbacks_config)
    callbacks_list = model_trainer_callbacks_config.get_tensorboard_and_checkpoint()
    
    model_trainer_config = config.get_model_training_config()
    model_trainer_config = Training(config=model_trainer_config)
    model_trainer_config.get_base_model()
    model_trainer_config.train_valid_generator()
    model_trainer_config.train(callbacks_list=callbacks_list)
except Exception as e:
    raise e

[2023-07-11 19:02:41,567: INFO: common: yaml file: config.yaml loaded successfully]
[2023-07-11 19:02:41,570: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-11 19:02:41,571: INFO: common: created directory at: artifacts]
[2023-07-11 19:02:41,573: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2023-07-11 19:02:41,574: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2023-07-11 19:02:41,575: INFO: common: created directory at: artifacts\training]
Found 3667 images belonging to 10 classes.
Found 14678 images belonging to 10 classes.
Epoch 1/10
917/917 [==============================] - 1166s 1s/step - loss: 7.9407 - recall: 0.5679 - val_loss: 4.6046 - val_recall: 0.7350
Epoch 2/10


c:\Users\Admin\anaconda3\envs\tomato\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


917/917 [==============================] - 1185s 1s/step - loss: 6.8384 - recall: 0.6859 - val_loss: 5.1191 - val_recall: 0.7650
Epoch 3/10
917/917 [==============================] - 1190s 1s/step - loss: 6.9140 - recall: 0.7200 - val_loss: 6.8134 - val_recall: 0.7385
Epoch 4/10
917/917 [==============================] - ETA: 0s - loss: 6.8643 - recall: 0.7381

KeyboardInterrupt: 